# Appendix 10.2.4: Tool choice

The Claude API supports a parameter called `tool_choice` that allows you to specify how you want Claude to call tools. In this notebook, we'll take a look at how it works and when to use it.

When working with the `tool_choice` parameter, we have three possible options: 

* `auto` allows Claude to decide whether to call any provided tools or not.
* `any` tells Claude that it must use one of the provided tools, but doesn't force a particular tool.
* `tool` allows us to force Claude to always use a particular tool.


This diagram illustrates how each option works: 

![tool_choice.png](./images/tool_choice.png)

Let's take a look at each option in detail. We'll start by importing the Anthropic SDK:

In [1]:
%pip install -qU pip
%pip install -qUr requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

In [3]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print(f'Using region: ', {region})

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

Using modelId: anthropic.claude-3-sonnet-20240229-v1:0
Using region:  {'us-west-2'}


## Auto

Setting `tool_choice` to `auto` allows the model to automatically decide whether to use tools or not.  This is the default behavior when working with tools if you don't use the `tool_choice` parameter at all.

To demonstrate this, we're going to provide Claude with a fake web search tool. We will ask Claude questions, some of which would require calling the web search tool and others which Claude should be able to answer on its own.

Let's start by defining a tool called `web_search`.  Please note, to keep this demo simple, we're not actually searching the web here.

We also set `toolChoice` to `auto`.

In [4]:
def web_search(topic):
    print(f"pretending to search the web for {topic}")

toolConfig = {'tools': [],
        "toolChoice": {
        "auto":{},
    }
}

toolConfig['tools'].append({
      "toolSpec": {
        "name": "web_search",
        "description": "A tool to retrieve up to date information on a given topic by searching the web. Only search the web for queries that you can not confidently answer.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "topic": {"type": "string", "description": "The topic to search the web for"}
            },
            "required": ["topic"]
          }
        }
      }
    })

Next, we write a function that accepts a `user_query` and passes it along to Claude, along with the `web_search_tool`. 

Here's the complete function:

In [5]:
from datetime import date

def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": [{"text": user_query}]}]

    system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.  
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)

    stop_reason = response['stopReason']

    if stop_reason == "end_turn":
        print("Claude did NOT call a tool")
        print(f"Assistant: {stop_reason}")
    elif stop_reason == "tool_use":
        print("Claude wants to use a tool")
        print(stop_reason)

Let's start with a question Claude should be able to answer without using the tool:

In [6]:
chat_with_web_search("What color is the sky?")

Claude did NOT call a tool
Assistant: end_turn


When we ask "What color is the sky?", Claude does not use the tool.  Let's try asking something that Claude should use the web search tool to answer:

In [7]:
chat_with_web_search("Who won the 2024 Miami Grand Prix?")

Claude wants to use a tool
tool_use


When we ask "Who won the 2024 Miami Grand Prix?", Claude uses the web search tool! 

Let's try a few more examples:

In [8]:
# Claude should NOT need to use the tool for this:
chat_with_web_search("Who won the Superbowl in 2022?")

Claude did NOT call a tool
Assistant: end_turn


In [9]:
# Claude SHOULD use the tool for this:
chat_with_web_search("Who won the Superbowl in 2024?")

Claude wants to use a tool
tool_use


### Your prompt matters!

When working with `toolChoice` set to `auto`, it's important that you spend time to write a detailed prompt.  Often, Claude can be over-eager to call tools.  Writing a detailed prompt helps Claude determine when to call a tool and when not to.  In the above example, we included specific instructions in the system prompt:

In [10]:
system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
"""

***

## Forcing a specific tool

We can force Claude to use a particular tool using `toolChoice`.  In the example below, we've defined two simple tools: 
* `print_sentiment_scores` - a tool that "tricks" Claude into generating well-structured JSON output containing sentiment analysis data.  For more info on this approach, see [Extracting Structured JSON using Claude and Tool Use](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/extracting_structured_json.ipynb) in the Anthropic Cookbook.
* `calculator` - a very simple calculator tool that takes two numbers and adds them together .


Our goal is to write a function called `analyze_tweet_sentiment` that takes in a tweet and uses Claude to print a basic sentiment analysis of that tweet.  Eventually we will "force" Claude to use the `print_sentiment_scores` tool, but we'll start by showing what happens when we **do not** force the tool use. 

In this first "bad" version of the `analyze_tweet_sentiment` function, we provide Claude with both tools. For the sake of comparison, we'll start by setting `toolChoice` to `auto`:

In [11]:
# Create our toolConfig
toolConfig = {'tools': [],
        "toolChoice": {
        "auto":{},
    }
}

# append our print_sentiment_scores tool
toolConfig['tools'].append({
    "toolSpec": {
      "name": "print_sentiment_scores",
      "description": "Prints the sentiment scores of a given tweet or piece of text.",
      "inputSchema": {
        "json": {
          "type": "object",
          "properties": {
            "positive_score": {"type": "number","description": "The positive sentiment score, ranging from 0.0 to 1.0."},
            "negative_score": {"type": "number","description": "The negative sentiment score, ranging from 0.0 to 1.0."},
            "neutral_score": {"type": "number","description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
          },
          "required": ["positive_score", "negative_score", "neutral_score"]
        }
      }
    }
  })

# Append our Calculator tool
toolConfig['tools'].append({
    "toolSpec": {
      "name": "calculator",
      "description": "Adds two numbers",
      "inputSchema": {
        "json": {
          "type": "object",
          "properties": {
            "num1": {"type": "number", "description": "first number to add"},
            "num2": {"type": "number", "description": "second number to add"}
          },
          "required": ["num1", "num2"]
        }
      }
    }
  })

Please note that we are deliberately not providing Claude with a well-written prompt, to make it easier to see the impact of forcing the use of a particular tool.

In [12]:
def analyze_tweet_sentiment(query):
    messages = [{"role": "user", "content": [{"text": query}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }
    response = bedrock_client.converse(**converse_api_params)
    print(response)

Let's see what happens when we call the function with the tweet `Holy cow, I just made the most incredible meal!`

In [13]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

{'ResponseMetadata': {'RequestId': 'd0de4cb3-4f11-4586-bcfc-95f2aebeb49f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 28 Aug 2024 03:29:54 GMT', 'content-type': 'application/json', 'content-length': '494', 'connection': 'keep-alive', 'x-amzn-requestid': 'd0de4cb3-4f11-4586-bcfc-95f2aebeb49f'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "That's great to hear! I'm glad you enjoyed your meal so much. Since you didn't ask a specific question, I don't have enough context to provide a more substantive response. But I'd be happy to discuss your cooking experience further if you have any other details to share about the incredible meal you prepared."}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 495, 'outputTokens': 68, 'totalTokens': 563}, 'metrics': {'latencyMs': 2057}}


Claude does not call our `print_sentiment_scores` tool and instead responds directly with:
> "That's great to hear! I don't actually have the capability to assess sentiment from text, but it sounds like you're really excited and proud of the incredible meal you made

Next, let's imagine someone tweets this: `I love my cats! I had four and just adopted 2 more! Guess how many I have now?`

In [14]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

{'ResponseMetadata': {'RequestId': '4b95ff9b-c158-4beb-b9b6-e64f1c5b8076', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 28 Aug 2024 03:29:56 GMT', 'content-type': 'application/json', 'content-length': '442', 'connection': 'keep-alive', 'x-amzn-requestid': '4b95ff9b-c158-4beb-b9b6-e64f1c5b8076'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'No need to use any tools for this simple math problem. If you had 4 cats originally and adopted 2 more, then the total number of cats you have now is:'}, {'toolUse': {'toolUseId': 'tooluse_4UkesfluRqSdZqsluXghuA', 'name': 'calculator', 'input': {'num1': 4, 'num2': 2}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 508, 'outputTokens': 109, 'totalTokens': 617}, 'metrics': {'latencyMs': 2161}}


Claude wants to call the calculator tool:

> {'toolUse': {'toolUseId': 'tooluse_oyzX9vToT468sAwe_A99EA', **'name': 'calculator', 'input': {'num1': 4, 'num2': 2}**}}]}}, 'stopReason': 'tool_use'{'toolUse': {'toolUseId': 'tooluse_oyzX9vToT468sAwe_A99EA', 'name': 'calculator', 'input': {'num1': 4, 'num2': 2}}}]}}, 'stopReason': 'tool_use'

Clearly, this current implementation is not doing what we want (mostly because we set it up to fail). 

So let's force Claude to **always** use the `print_sentiment_scores` tool by updating `toolChoice`:

In [15]:
toolConfig['toolChoice'] = {
    "tool": {
        "name": "print_sentiment_scores"}
}

In addition to setting `type` to `tool`, we must provide a particular tool name.

In [16]:
def analyze_tweet_sentiment(query):
    messages = [{"role": "user", "content": [{"text": query}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }
    response = bedrock_client.converse(**converse_api_params)
    print(response)

Now if we try prompting Claude with the same prompts from earlier, it's always going to call the `print_sentiment_scores` tool:

In [17]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

{'ResponseMetadata': {'RequestId': '9453906d-44ee-4923-9ea0-35d94ac91e54', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 28 Aug 2024 03:29:59 GMT', 'content-type': 'application/json', 'content-length': '335', 'connection': 'keep-alive', 'x-amzn-requestid': '9453906d-44ee-4923-9ea0-35d94ac91e54'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_R3RfJCVgS3O81p2pfCJAxA', 'name': 'print_sentiment_scores', 'input': {'positive_score': 0.9, 'negative_score': 0.1, 'neutral_score': 0.0}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 593, 'outputTokens': 79, 'totalTokens': 672}, 'metrics': {'latencyMs': 3181}}


Claude calls our `print_sentiment_scores` tool:

> [{'toolUse': {'toolUseId': 'tooluse_EZnn27PHRXWfo7JR8FWkDw', **'name': 'print_sentiment_scores',** 'input': {'positive_score': 0.9, 'negative_score': 0.1, 'neutral_score': 0.0}}}][{'toolUse': {'toolUseId': 'tooluse_EZnn27PHRXWfo7JR8FWkDw', 'name': 'print_sentiment_scores', 'input': {'positive_score': 0.9, 'negative_score': 0.1, 'neutral_score': 0.0}}}]

Even if we try to trip up Claude with a "Math-y" tweet, it still always calls the `print_sentiment_scores` tool:

In [18]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

{'ResponseMetadata': {'RequestId': 'f1b43d8e-8908-4944-bd72-e7bc66a7108a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 28 Aug 2024 03:30:01 GMT', 'content-type': 'application/json', 'content-length': '335', 'connection': 'keep-alive', 'x-amzn-requestid': 'f1b43d8e-8908-4944-bd72-e7bc66a7108a'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_zu8yfPb5QSeo0mIkqN420A', 'name': 'print_sentiment_scores', 'input': {'positive_score': 0.8, 'negative_score': 0.1, 'neutral_score': 0.1}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 606, 'outputTokens': 79, 'totalTokens': 685}, 'metrics': {'latencyMs': 1545}}


Even though we're forcing Claude to call our `print_sentiment_scores` tool, we should still employ some basic prompt engineering to give Claude better task context:

In [19]:
def analyze_tweet_sentiment(query):
    prompt = f"""
    Analyze the sentiment in the following tweet:
    <tweet>{query}</tweet>"""

    messages = [{"role": "user", "content": [{"text": prompt}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }
    response = bedrock_client.converse(**converse_api_params)
    print(response)

***

## Any

The final option for `toolChoice` is `any`, which allows us to tell Claude, "You must call a tool, but you can pick which one."  Imagine we want to create a SMS chatbot using Claude.  The only way for this chatbot to actually "communicate" with a user is via SMS text message. 

In the example below, we make a very simple text-messaging assistant that has access to two tools:
* `send_text_to_user` - sends a text message to a user.
* `get_customer_info` - looks up customer data based on a username.

The idea is to create a chatbot that always calls one of these tools and never responds with a non-tool response.  In all situations, Claude should either respond back by trying to send a text message or calling `get_customer_info` to get more customer information. To ensure this, we set `toolChoice` to `any`:

In [20]:
toolConfig = {'tools': [],
        "toolChoice": {
        "any":{},
    }
}

toolConfig['tools'].append({
      "toolSpec": {
        "name": "send_text_to_user",
        "description": "Sends a text message to a user",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "text": {
                "type": "string",
                "description": "The piece of text to be sent to the user via text message"}
            },
            "required": ["text"]
          }
        }
      }
    })

toolConfig['tools'].append({
      "toolSpec": {
        "name": "get_customer_info",
        "description": "gets information on a customer based on the customer's username.  Response includes email, username, and previous purchases. Only call this tool once a user has provided you with their username",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "username": {
                "type": "string",
                "description": "The username of the user in question. "}
            },
            "required": ["username"]
          }
        }
      }
    })

In [21]:
#toolConfig # Optional uncomment to see the updated toolConfig

In [22]:
def send_text_to_user(text):
    # Sends a text to the user
    # We'll just print out the text to keep things simple:
    print(f"TEXT MESSAGE SENT: {text}")

def get_customer_info(username):
    return {
        "username": username,
        "email": f"{username}@email.com",
        "purchases": [
            {"id": 1, "product": "computer mouse"},
            {"id": 2, "product": "screen protector"},
            {"id": 3, "product": "usb charging cable"},
        ]
    }

system_prompt = """
All your communication with a user is done via text message.
Only call tools when you have enough information to accurately call them.  
Do not call the get_customer_info tool until a user has provided you with their username. This is important.
If you do not know a user's username, simply ask a user for their username.
"""

def sms_chatbot(user_message):
    messages = [{"role": "user", "content": [{"text": user_message}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }

    response = bedrock_client.converse(**converse_api_params)

    if(response['stopReason'] == "tool_use"):
        tool_use = response['output']['message']['content'][-1]
        tool_name = tool_use['toolUse']['name']
        tool_inputs = tool_use['toolUse']['input']
        print(f"=======Claude Wants To Call The {tool_name} Tool=======")
        if tool_name == "send_text_to_user":
            send_text_to_user(tool_inputs["text"])
        elif tool_name == "get_customer_info":
            print(get_customer_info(tool_inputs["username"]))
        else:
            print("Oh dear, that tool doesn't exist!")
            
    else:
        print("No tool was called. This shouldn't happen!")

Let's start simple:

In [23]:
sms_chatbot("Hey there! How are you?")

=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: Hello! I'm an AI assistant. How can I help you today?


Claude responds back by calling the `send_text_to_user` tool.

Next, we'll ask Claude something a bit trickier:

In [24]:
sms_chatbot("I need help looking up an order")

=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: Sure, I can help you look up your order information. To get started, please provide me with your username so I can retrieve your account details.


Claude wants to send a text message, asking a user to provide their username.

Now, let's see what happens when we provide Claude with our username:

In [25]:
sms_chatbot("I need help looking up an order.  My username is jenny76")

=======Claude Wants To Call The get_customer_info Tool=======
{'username': 'jenny76', 'email': 'jenny76@email.com', 'purchases': [{'id': 1, 'product': 'computer mouse'}, {'id': 2, 'product': 'screen protector'}, {'id': 3, 'product': 'usb charging cable'}]}


Claude calls the `get_customer_info` tool, just as we hoped! 

Even if we send Claude a gibberish message, it will still call one of our tools:

In [26]:
sms_chatbot("askdj aksjdh asjkdbhas kjdhas 1+1 ajsdh")

=======Claude Wants To Call The send_text_to_user Tool=======
TEXT MESSAGE SENT: I'm afraid I don't understand your query. Could you please rephrase it or provide more context? I'll do my best to assist you once I understand what you need.
